## Project: Redbus Data Scraping with Selenium & Dynamic Filtering using Streamlit

### Web scraping using selenium

#### 1. Installed required packages

In [ ]:
# pip install selenium
# pip install streamlit
# pip install pandas
# pip install mysql-connector-python
# pip install numpy

#### 2. Imported required packages

In [14]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import mysql.connector
import numpy as np
import os

#### 3. Extracting state bus links using selenium webdriver in edge browser

In [ ]:
# setting up the driver
driver = webdriver.Edge()
driver.get('https://www.redbus.in')
driver.maximize_window()
time.sleep(0.5)

# finding the links from the source
redbus_links = driver.find_element(By.XPATH,"//*[@id='homeV2-root']/div[3]/div[1]/div[2]/a")
redbus_links.click()

# Wait for the new tab to open
time.sleep(0.5)

# Fetch the URL of the new tab
c_url = redbus_links.get_attribute('href')
driver.get(c_url)

# Extracting the links from the new tab
links=driver.find_elements(By.CLASS_NAME,'D113_link')
links_webs={}
for i in links:
    links_webs.update({i.text: i.get_attribute('href')})

# Close the browser
driver.quit()

links_webs

#### 4. Scraping all the states and storing the data in a csv file

In [12]:
# Initialize the Edge driver
driver = webdriver.Edge()

# Dictionary of state links
state_link_list = {
    'KSRTC (Kerala)': 'https://www.redbus.in/online-booking/ksrtc-kerala',
    'APSRTC': 'https://www.redbus.in/online-booking/apsrtc',
    'TSRTC': 'https://www.redbus.in/online-booking/tsrtc',
    'RSRTC': 'https://www.redbus.in/online-booking/rsrtc',
    'PePSU (Punjab)': 'https://www.redbus.in/online-booking/pepsu-punjab',
    'HRTC': 'https://www.redbus.in/online-booking/hrtc',
    'WBTC (CTC)': 'https://www.redbus.in/online-booking/wbtc-ctc',
    'ASTC': 'https://www.redbus.in/online-booking/astc',
    'KTCL': 'https://www.redbus.in/online-booking/ktcl',
    'BSRTC': 'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc'
}

# Function to scrape route links and bus information for a specific state
def scrape_state_buses(state_name, state_url):
    # State bus data dictionary to store the scraped data
    state_dict = {
        "State": [], "Route_name": [], "Bus_name": [], "Bus_type": [], "Departure_time": [],
        "Duration": [], "Arrival_time": [], "Ratings": [], "Fare": [], "Seats_available": [], "Link": []
    }

    # Access the state buses page
    driver.get(state_url)
    time.sleep(0.5) 

    wait = WebDriverWait(driver, 20)
    state_links_list = []

    # Pagination to collect route links
    page_number = 1
    while True:
        try:
            # Collect all route links on the current page
            state_links = driver.find_elements(By.XPATH, '//a[@class="route"]')
            for route in state_links:
                state_links_list.append(route.get_attribute('href'))
                state_dict["Route_name"].append(route.text)
                state_dict["Link"].append(route.get_attribute('href'))

            # Try to click the next page button
            try:
                next_page_button = driver.find_element(By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']")
                action = ActionChains(driver)
                action.move_to_element(next_page_button).perform()
                time.sleep(2)
                next_page_button.click()
                page_number += 1
                time.sleep(3)
            except NoSuchElementException:
                print(f"No more pages found for {state_name}.")
                break  # Exit the loop if no more pages are found
        except Exception as e:
            print(f"Error in scraping route links for {state_name}: {e}")
            break

    # Get unique route links
    state_unique_links = list(set(state_links_list))
    print(f"Number of unique routes for {state_name}: {len(state_unique_links)}")

    # Iterate through each unique route link to get bus details
    for route_link in state_unique_links:
        driver.get(route_link)
        time.sleep(3)

        # Try to click "View Buses" for additional buses (if available)
        try:
            view_buses_buttons = driver.find_elements(By.XPATH, '//div[@class="button"]')
            for button in reversed(view_buses_buttons):
                button.click()
                time.sleep(2)
        except:
            pass
        
        # Infinite scrolling to load all bus details on the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Scrape bus data
        buses = driver.find_elements(By.XPATH, "//div[@class='clearfix bus-item-details']")
        State_route_name = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]').text
        State_route_link = route_link

        for bus in buses:
            # Ensure that every key has a value even if it is missing
            state_dict['State'].append(state_name)
            state_dict['Route_name'].append(State_route_name)
            state_dict['Link'].append(State_route_link)

            try:
                state_dict['Bus_name'].append(bus.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text)
            except:
                state_dict['Bus_name'].append(None)

            try:
                state_dict['Bus_type'].append(bus.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text)
            except:
                state_dict['Bus_type'].append(None)

            try:
                state_dict['Departure_time'].append(bus.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text)
            except:
                state_dict['Departure_time'].append(None)

            try:
                state_dict['Duration'].append(bus.find_element(By.XPATH, ".//div[@class='column-four p-right-10 w-10 fl']").text)
            except:
                state_dict['Duration'].append(None)

            try:
                state_dict['Arrival_time'].append(bus.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text)
            except:
                state_dict['Arrival_time'].append(None)

            try:
                state_dict['Ratings'].append(float(bus.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text))
            except:
                state_dict['Ratings'].append(0)

            try:
                state_dict['Fare'].append(bus.find_element(By.XPATH, ".//span[@class='f-19 f-bold']").text)
            except:
                state_dict['Fare'].append(0)

            try:
                state_dict['Seats_available'].append(bus.find_element(By.XPATH, ".//div[@class='seat-left m-top-16']").text[0:2])
            except:
                state_dict['Seats_available'].append(0)

    # Ensure all lists in the dictionary are of the same length
    min_length = min(len(v) for v in state_dict.values())
    for key in state_dict:
        state_dict[key] = state_dict[key][:min_length]

    # Convert the scraped data to a DataFrame
    df = pd.DataFrame(state_dict)

    # Data type conversions
    df['Fare'] = df['Fare'].replace('', np.nan)
    df['Fare'] = pd.to_numeric(df['Fare'], errors='coerce').fillna(0).astype(float)

    # Clean time strings and convert to time format
    df['Departure_time'] = df['Departure_time'].apply(lambda x: x.strip() + ':00' if isinstance(x, str) and len(x.strip()) <= 2 else x)
    df['Arrival_time'] = df['Arrival_time'].apply(lambda x: x.strip() + ':00' if isinstance(x, str) and len(x.strip()) <= 2 else x)

    # Convert to time format
    df['Departure_time'] = pd.to_datetime(df['Departure_time'], format='%H:%M', errors='coerce').dt.time
    df['Arrival_time'] = pd.to_datetime(df['Arrival_time'], format='%H:%M', errors='coerce').dt.time

    # Save the data to a CSV file for the state
    df.to_csv(f'{state_name}_bus_data.csv', index=False)
    print(f"Data saved to {state_name}_bus_data.csv")

# Loop through each state and scrape the bus data
for state_name, state_url in state_link_list.items():
    scrape_state_buses(state_name, state_url)

# Close the browser after scraping all states
driver.quit()


No more pages found for KSRTC (Kerala).
Number of unique routes for KSRTC (Kerala): 15
Data saved to KSRTC (Kerala)_bus_data.csv
No more pages found for APSRTC.
Number of unique routes for APSRTC: 46
Data saved to APSRTC_bus_data.csv
No more pages found for TSRTC.
Number of unique routes for TSRTC: 26
Data saved to TSRTC_bus_data.csv
No more pages found for RSRTC.
Number of unique routes for RSRTC: 18
Data saved to RSRTC_bus_data.csv
No more pages found for PePSU (Punjab).
Number of unique routes for PePSU (Punjab): 18
Data saved to PePSU (Punjab)_bus_data.csv
No more pages found for HRTC.
Number of unique routes for HRTC: 35
Data saved to HRTC_bus_data.csv
No more pages found for WBTC (CTC).
Number of unique routes for WBTC (CTC): 38
Data saved to WBTC (CTC)_bus_data.csv
No more pages found for ASTC.
Number of unique routes for ASTC: 47
Data saved to ASTC_bus_data.csv
No more pages found for KTCL.
Number of unique routes for KTCL: 34
Data saved to KTCL_bus_data.csv
No more pages found

#### 5. Separating the from_city an₫ to_city from the Route_name

In [17]:
# Define the folder where your CSV files are stored
csv_folder = r'H:\01-My Passport\05-Education_Learning\06_Guvi\Data Science\Mini Projects\01_Redbus_Transport\02_final_project\131024_redbus_csv_files'

# Function to split route name into "From" and "To" cities
def split_route(route_name):
    if " to " in route_name:
        # Split on " to " to separate From and To
        from_city, to_city = route_name.split(" to ")
        return from_city.strip(), to_city.strip()  # Return cleaned 'From' and 'To' cities
    else:
        return None, None  # If format is incorrect, return None

# Process each CSV file in the folder
for file in os.listdir(csv_folder):
    if file.endswith('.csv'):
        print(f"Processing {file}...")
        
        # Full path to the CSV file
        csv_file = os.path.join(csv_folder, file)
        
        # Load the CSV file into a DataFrame
        df = pd.read_csv(csv_file)
        
        # Ensure 'Route_name' exists in the CSV
        if 'Route_name' in df.columns:
            # Apply the split_route function to split 'Route_name' into 'From' and 'To'
            df['From_City'], df['To_City'] = zip(*df['Route_name'].apply(split_route))
            
            # Save the updated DataFrame back to a new CSV file (or overwrite the existing file)
            output_file = os.path.join(csv_folder, f"01_FAT_{file}")
            df.to_csv(output_file, index=False)
            
            print(f"File saved as {output_file}")

print("All files processed and saved.")

Processing APSRTC_bus_data.csv...
File saved as H:\01-My Passport\05-Education_Learning\06_Guvi\Data Science\Mini Projects\01_Redbus_Transport\02_final_project\131024_redbus_csv_files\01_FAT_APSRTC_bus_data.csv
Processing ASTC_bus_data.csv...
File saved as H:\01-My Passport\05-Education_Learning\06_Guvi\Data Science\Mini Projects\01_Redbus_Transport\02_final_project\131024_redbus_csv_files\01_FAT_ASTC_bus_data.csv
Processing BSRTC_bus_data.csv...
File saved as H:\01-My Passport\05-Education_Learning\06_Guvi\Data Science\Mini Projects\01_Redbus_Transport\02_final_project\131024_redbus_csv_files\01_FAT_BSRTC_bus_data.csv
Processing HRTC_bus_data.csv...
File saved as H:\01-My Passport\05-Education_Learning\06_Guvi\Data Science\Mini Projects\01_Redbus_Transport\02_final_project\131024_redbus_csv_files\01_FAT_HRTC_bus_data.csv
Processing KSRTC (Kerala)_bus_data.csv...
File saved as H:\01-My Passport\05-Education_Learning\06_Guvi\Data Science\Mini Projects\01_Redbus_Transport\02_final_projec

#### 6.0 Connecting to the MySQL server and creating the database, then adding the data

In [21]:
# Connect to MySQL
mydb = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = "Pnu@2024",
  database = "redbus_data"
)

mycursor = mydb.cursor()

# Create the table if it doesn't exist
mycursor.execute("""
CREATE TABLE IF NOT EXISTS redbus_details (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    From_City VARCHAR(255),
    To_City VARCHAR(255),
    Route_name VARCHAR(255),
    Bus_name VARCHAR(255),
    Bus_type VARCHAR(255),
    Departure_time TIME,
    Duration VARCHAR(50),
    Arrival_time TIME,
    Ratings DECIMAL(3, 2),
    Fare DECIMAL(10, 2),
    Seat_availability INT,
    Link TEXT
    
);
""")

def insert_into_db(dataframe):
    # Replace NaN values with None
    dataframe = dataframe.where(pd.notnull(dataframe), None)
    
    for index, row in dataframe.iterrows():
        sql = """
        INSERT INTO redbus_details (State, From_City, To_City, Route_name, Bus_name, Bus_type, Departure_time, Duration, Arrival_time, Ratings, Fare, Seat_availability, Link)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (
            row['State'], row['From_City'], row['To_City'], row['Route_name'], row['Bus_name'], row['Bus_type'],
            row['Departure_time'], row['Duration'], row['Arrival_time'], row['Ratings'], row['Fare'], row['Seats_available'], row['Link']
        )

        # Log values before execution for debugging purposes
        print("Inserting values:", values)
        
        # Execute SQL with the provided values
        mycursor.execute(sql, values)
    
    # Commit the transaction after inserting the data
    mydb.commit()

# Specify the folder containing the CSV files
csv_folder = r'H:\01-My Passport\05-Education_Learning\06_Guvi\Data Science\Mini Projects\01_Redbus_Transport\02_final_project\131024_redbus_csv_files'

# Process each CSV file in the folder
for file in os.listdir(csv_folder):
    if file.endswith('.csv'):
        print(f"Processing {file}...")
        csv_file = os.path.join(csv_folder, file)
        df = pd.read_csv(csv_file)

        # Insert the CSV data into MySQL
        insert_into_db(df)

print("Data inserted successfully into MySQL.")

# Close the cursor and database connection
mycursor.close()
mydb.close()

Processing 01_FAT_APSRTC_bus_data.csv...
Inserting values: ('APSRTC', 'Hyderabad', 'Vijayawada', 'Hyderabad to Vijayawada', 'APSRTC - 2674', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', '17:30:00', '07h 50m', '01:20:00', 2.6, 436.0, 0, 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada')
Inserting values: ('APSRTC', 'Vijayawada', 'Hyderabad', 'Vijayawada to Hyderabad', 'APSRTC - 3366', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', '17:55:00', '07h 50m', '01:45:00', 2.4, 436.0, 0, 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad')
Inserting values: ('APSRTC', 'Hyderabad', 'Ongole', 'Hyderabad to Ongole', 'APSRTC - 3360', 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', '18:25:00', '07h 05m', '01:30:00', 2.6, 720.0, 0, 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole')
Inserting values: ('APSRTC', 'Kakinada', 'Visakhapatnam', 'Kakinada to Visakhapatnam', 'APSRTC - 2575', 'NIGHT RIDER (SEATER CUM SLEEPER)', '18:55:00', '06h 20m', '01:15:00', 4.0, 595.0, 0, 'https://www.redbu